# Making dataframe with all metrics for project

In [1]:
import pandas as pd
import numpy as np
import os
from collections import defaultdict

#from statsmodels.stats.multitest import fdrcorrection

current_dir = os.getcwd()

In [2]:
annot = pd.read_csv(os.path.join(current_dir, '..', 'data', 'annotation', 'hg19_genes_zero_based.txt'), sep='\t',
                    names=['chr', 'start', 'end', 'strand', 'gene_name', 'gene_type'])

### Step 1. gerp+phast+phylo data

**Goal**: From the files obtained after bedtools map with the calculated sum of signals per gene, obtain a data frame with the average value of the metric per gene.

In [83]:
#get current working dir and check files in folder
current_dir = os.getcwd()
os.listdir(os.path.join(current_dir, '..', 'data', 'gerp_phast_phylo'))

['gerp_500_RS_by_gene.bed',
 'gerp_RS_by_gene.bed',
 'phastCons_500_by_gene.bed',
 'phastCons_by_gene.bed',
 'phyloP_500_by_gene.bed',
 'phyloP_by_gene.bed']

In [20]:
#read columns with values in dfs
files_info = {
    'gerp_RS_by_gene.bed': 'RS_value',
    'gerp_500_RS_by_gene.bed': 'RS_500_value',
    'phastCons_by_gene.bed': 'phast_value',
    'phastCons_500_by_gene.bed': 'phast_500_value',
    'phyloP_by_gene.bed': 'phylo_value',
    'phyloP_500_by_gene.bed': 'phylo_500_value'}

col_names = ['chr', 'start', 'end', 'gene_name', 'sum_value']
dfs = pd.DataFrame()

for file_name, new_col_name in files_info.items():
    file_path = os.path.abspath(os.path.join(current_dir, '..', 'data', 'gerp_phast_phylo', file_name))
    df = pd.read_csv(file_path, sep='\t', names=col_names)
    df = df.rename(columns={'sum_value': new_col_name})  
    dfs[new_col_name] = df[new_col_name]

In [21]:
#change dtypes
dfs = dfs.replace(to_replace='.', value=np.nan)
dfs = dfs.astype(dtype='float64')

#add features
genes_data = pd.read_csv(os.path.join(current_dir, '..', 'data', 'annotation', 'hg19_genes_zero_based.txt'),
                         sep='\t', header=None, names=['chr', 'start', 'end', 'strand', 'gene_name', 'gene_type'])

g_data = genes_data[['chr', 'start', 'end', 'gene_name', 'gene_type']]
gpp_all_genes = pd.concat([g_data, dfs], axis=1)

In [22]:
#add gene lenght and change column order
gpp_all_genes['gene_len'] = gpp_all_genes['end'] - gpp_all_genes['start']
gpp_all_genes['gene_len_500'] = (gpp_all_genes['end'] + 500) - (gpp_all_genes['start'] - 500)

gpp_all_genes = gpp_all_genes[['chr', 'start', 'end', 'gene_name', 'gene_type', 
                                'gene_len', 'gene_len_500', 'RS_value', 'RS_500_value', 
                                'phast_value', 'phast_500_value', 'phylo_value', 'phylo_500_value']]

In [23]:
#count mean for metrics
value_columns = ['RS_value', 'phast_value', 'phylo_value']
for col in value_columns:
    gpp_all_genes[col] = gpp_all_genes[col] / gpp_all_genes['gene_len']

value_columns_500 = ['RS_500_value', 'phast_500_value', 'phylo_500_value']
for col in value_columns_500:
    gpp_all_genes[col] = gpp_all_genes[col] / gpp_all_genes['gene_len_500']

In [ ]:
#check result
gpp_all_genes

In [25]:
#save ucsc metrics in `gpp_data_all_genes.tsv` (GerpPhastPhylo = gpp)
file_name_gpp = 'gpp_data_all_genes.tsv'
file_path_to_gpp = os.path.abspath(os.path.join(current_dir, '..', 'data', 'merged_data', file_name_gpp))
gpp_all_genes.to_csv(file_path_to_gpp, sep='\t', index=False, header=True)

### Step 2. iHS data extraction and preparation

Strange data format:

DAF = derived allele frequency

stdiHS = standardized iHS score

populations = ESN|GWD|LWK|MSL|YRI|ACB|ASW|CLM|MXL|PEL|PUR|CDX|CHB|CHS|JPT|KHV|CEU|FIN|GBR|IBS|TSI|BEB|GIH|ITU|PJL|STU

```bash
CHR	POS	RSNUM	DAF	stdIHS
21	9509809	rs370181688	[DAF for all 26 populations, sep='|']\t[stdIHS for all 26 populations, sep='|']
```

Data processing via `2.1_iHS_data_processing.ipynb` script to get files with DAF and stdiHS for each chromosome

In [ ]:
#modified script to search for the first maximum among all signal values in a certain position
#get only standardized iHS metrics from each file
'''
ihs_files = os.listdir("D:\\iHS\\iHS_first_max")
out_folder = "D:\\iHS\\iHS_max_formatted_bed"
os.makedirs(out_folder, exist_ok=True)

for file in ihs_files:
    file_path = os.path.join("D:\\iHS\\iHS_first_max\\", file)
    df = pd.read_csv(file_path, sep=',', low_memory=False)
    df['chr'] = 'chr' + df['CHR'].astype(str)
    df['start'] = df['POS'] - 1
    df['end'] = df['POS']
    df['max_stdiHS'] = df['max_stdiHS'].fillna(0)
    final_data = df[['chr', 'start', 'end', 'max_stdiHS']]
    
    base_name = os.path.splitext(file)[0]
    prefix = base_name.split(".")[0]
    output = os.path.join(out_folder, f"{prefix}_iHS_metrics.bed")
    final_data.to_csv(output, index=False, sep='\t', header=False)
'''

After getting stdiHS we need to merge all the files with data for each chromosome into one common file:

```bash
cat *.bed > iHS_stdiHS_merged.bed
```
Check result:
```bash
awk '{print $1}' iHS_stdiHS_merged.bed | uniq
```
Sort:
```bash
sort -k1,1 -k2,2n iHS_stdiHS_merged.bed > iHS_std_sorted.bed
```
Run `bedtools map` to obtain max value on gene level:
```bash
bedtools map -o max -a hg19_0_based_500.bed -b iHS_std_sorted.bed -c 4 > iHS_max_by_gene_500.bed
```

After all steps we need to concat gpp with iHS data

In [11]:
#read data and replace missing values ('.') with np.nan
col_names = ['chr', 'start', 'end', 'gene_name', 'iHS_GBR_max_500']

dfs = pd.read_csv(os.path.join(current_dir, '..', 'data', 'iHS', 'iHS_max_GBR_by_gene_500.bed'),
                  sep='\t', header=None, names=col_names)

dfs = dfs.replace(to_replace='.', value=np.nan)
dfs['iHS_GBR_max_500'] = dfs['iHS_GBR_max_500'].astype(dtype='float64')

#read gpp data
gpp = pd.read_csv(os.path.join(current_dir, '..', 'data', 'merged_data', 'gpp_data_all_genes.tsv'), sep='\t')

In [31]:
#get dataframe to concat and count p-values
col_names_snp = ['chr', 'start', 'end', 'gene_name', 'iHS_snps_per_gene']

counts = pd.read_csv(os.path.join(current_dir, '..', 'data', 'iHS', 'iHS_counts_per_gene_500.bed'),
                  sep='\t', header=None, names=col_names_snp)

counts = counts.replace(to_replace='.', value=np.nan)
counts['iHS_snps_per_gene'] = counts['iHS_snps_per_gene'].astype(dtype='float64')

counts = counts.join(dfs['iHS_GBR_max_500'])

Count p-values for iHS (and fdr correction) and expected iHS

**Count pvals for iHS_max**

However, as indicated in the [thread](https://stats.stackexchange.com/questions/381212/distribution-of-maximum-of-normally-distributed-random-variables), we can do even better by taking the actual probability (p-value) of a given gene-wise maximum iHS value. 

The CDF (P(max(Y) < y)) = P(X1 < y) * … * P(Xn < y) [in other words, the maximum of n random variables is less than a given number y if value of each random variable are less than y]. 

In our case, we need P(max(Y) >= y) rather than P(max(Y) < y), so we can compute the p-value as p = 1 - pnorm(gene_max_ihs)^n_snp

In [51]:
from scipy.stats import stats, norm

#expected iHS
counts['expected_iHS'] = norm.ppf(1 - (1 / counts['iHS_snps_per_gene']))
counts['expected_iHS'] = counts['expected_iHS'].replace(to_replace=-np.inf, value=np.nan)

#pval
mask = counts[['iHS_GBR_max_500', 'iHS_snps_per_gene']].isna().any(axis=1)
counts['pval_iHS_max'] = np.where(mask, np.nan, 
                                  1 - (norm.cdf(counts['iHS_GBR_max_500']) ** counts['iHS_snps_per_gene']))

#pval with fdr correction
from scipy import stats
pval_mask = counts['pval_iHS_max'].notna()
fdr_corrected = stats.false_discovery_control(
    ps=counts.loc[pval_mask, 'pval_iHS_max'], 
    method='bh')

counts['pval_iHS_max_fdr'] = np.nan  
counts.loc[pval_mask, 'pval_iHS_max_fdr'] = fdr_corrected

In [81]:
#check correctness 
counts_0 = counts[counts['iHS_GBR_max_500'].isna()]
counts_0.pval_iHS_max_fdr.describe()

count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
Name: pval_iHS_max_fdr, dtype: float64

In [84]:
#merging gpp and iHS
merged_df = pd.concat([gpp, counts[['iHS_GBR_max_500', 'expected_iHS', 'iHS_snps_per_gene', 'pval_iHS_max', 'pval_iHS_max_fdr']]], axis=1)

#save file with gpp and iHS
merged_df.to_csv(os.path.join(current_dir, '..', 'data', 'merged_data', 'gpp_iHS_GBR.tsv'), sep='\t', index=False, header=True)

### Step 3. Merge gpp+iHS with 

- gnomAD

- DRC150

- SDS

- number of GO terms for each gene (BP, MF) 

- number of protein-protein interactions (from BioGRID database)

- GTEx data

In [18]:
#read gnomAD data 
df_gnomad = pd.read_csv(os.path.join(current_dir, '..', 'data', 'gnomAD', 'gnomad_metrics.csv'), sep=',')

#read gpp+iHS data
gpp_ihs = pd.read_csv(os.path.join(current_dir, '..', 'data', 'merged_data', 'gpp_iHS_GBR.tsv'), sep='\t')

#read GO data
go = pd.read_csv(os.path.abspath(os.path.join(current_dir, '..', 'data', 'go_analysis',
                                            'go_all_genes_counts_bp_mf.csv')), sep=',')

#read BioGRID data
biogrid = pd.read_csv(os.path.join(current_dir, '..', 'data', 'biogrid',
                    'biogrid_n_interactions_per_gene.tsv'), sep='\t')

Add data of positive selection signals: [DRC150](https://www.nature.com/articles/s41588-018-0177-x) and [SDS values](https://www.science.org/doi/10.1126/science.aag0776) 

In [26]:
#read drc150
path_to_drc150 = os.path.abspath(os.path.join(current_dir, '..', 'data', 'drc150_positive_selection', 
                                              'DRC150_max_by_gene_500.bed'))
df_drc = pd.read_csv(path_to_drc150, sep='\t', names=['chr', 'start', 'end', 'gene_name', 'DRC150'])

df_drc = df_drc.replace(to_replace='.', value=np.nan)
df_drc['DRC150'] = df_drc['DRC150'].astype(dtype='float64')

#read sds data
path_to_sds = os.path.abspath(os.path.join(current_dir, '..', 'data', 'sds_positive_selection',
                                           'SDS_max_by_gene_500.bed'))
df_sds = pd.read_csv(path_to_sds, sep='\t', names=['chr', 'start', 'end', 'gene_name', 'SDS'])
df_sds = df_sds.replace(to_replace='.', value=np.nan)
df_sds['SDS'] = df_sds['SDS'].astype(dtype='float64')

Add number of significiant SNPs from raw DRC150 data (p-value with fdr < 0.05)

In [27]:
path_to_drc = os.path.join(current_dir, '..', 'data', 'drc150_positive_selection', 'drc150_FDR_signif_snp_counts_per_gene.bed')
drc150_snp_count = pd.read_csv(path_to_drc, sep='\t', 
                               names=['chr', 'start', 'end', 'gene_name', 'DRC150_signif_snp_count'])

Merging gpp_ihs with DRC and SDS

In [35]:
gpp_ihs_drc_sds= pd.concat([gpp_ihs, df_drc[['DRC150']], df_sds[['SDS']], drc150_snp_count['DRC150_signif_snp_count']], axis=1)
gpp_ihs_drc_sds

,chr,start,end,gene_name,gene_type,gene_len,gene_len_500,RS_value,RS_500_value,phast_value,...,phylo_value,phylo_500_value,iHS_GBR_max_500,expected_iHS,iHS_snps_per_gene,pval_iHS_max,pval_iHS_max_fdr,DRC150,SDS,DRC150_signif_snp_count
0,chr1,11868,14412,DDX11L1,pseudogene,2544,3544,-0.234778,-0.221300,0.105669,...,0.237890,0.240297,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0
1,chr1,14362,29806,WASH7P,pseudogene,15444,16444,-0.234015,-0.233031,0.060497,...,0.129629,0.108072,0.0,0.674490,4.0,0.937500,1.0,NaN,NaN,0
2,chr1,29553,31109,MIR1302-11,lincRNA,1556,2556,-0.187395,-0.189201,0.037335,...,-0.153312,-0.165173,0.0,NaN,1.0,0.500000,1.0,NaN,NaN,0
3,chr1,34553,36081,FAM138A,lincRNA,1528,2528,-0.214907,-0.200191,0.048410,...,0.003804,0.058636,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0
4,chr1,52472,54936,OR4G4P,pseudogene,2464,3464,-0.150520,-0.199254,0.077379,...,0.134280,0.056223,0.0,1.281552,10.0,0.999023,1.0,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57778,chrY,59330251,59343488,IL9R,protein_coding,13237,14237,-0.283951,-0.290134,0.060365,...,-0.078255,-0.082713,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0
57779,chrY,59336888,59358305,AJ271736.10,processed_transcript,21417,22417,-0.131159,-0.134897,0.079113,...,0.108522,0.110373,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0
57780,chrY,59347293,59349508,WASIR1,antisense,2215,3215,0.000000,0.000000,0.069377,...,-0.042003,-0.034462,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0
57781,chrY,59353496,59358381,WASH6P,pseudogene,4885,5885,-0.140084,-0.142133,0.136902,...,0.637514,0.534463,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0


Merging gpp_ihs_drc_sds with df_gnomAD

In [36]:
gpp_ihs_gnomad = pd.merge(left=gpp_ihs_drc_sds, right=df_gnomad, left_on='gene_name', 
                      right_on='gene', how='left')

Merging gpp_ihs_gnomad with GO data and BioGRID

In [40]:
gpp_ihs_gnomad_go = pd.merge(left=gpp_ihs_gnomad, right=go, on='gene_name', how='left')

In [60]:
gpp_ihs_gnomad_go_bg = pd.merge(left=gpp_ihs_gnomad_go, right=biogrid, on='gene_name', how='left')

In [67]:
print(f'Shape of merged data: {gpp_ihs_gnomad_go_bg.shape}')

Shape of merged data: (57874, 30)


After merging all data, extra rows appear. These are duplicates due to repeated gene names

In [68]:
#clear all duplicates based on start, end of gene
duplicates = gpp_ihs_gnomad_go_bg.duplicated(subset=['chr', 'gene_name', 'start', 'end'], keep=False)
print(f'Number of duplicates: {duplicates.sum()}')

#remove duplicates, leaving the first occurrence behind
filtered_data = gpp_ihs_gnomad_go_bg.drop_duplicates(subset=['chr', 'gene_name', 'start', 'end', 'gene_type'], keep='first')

#check result
print(f'Shape before filtering from duplicates: {gpp_ihs_gnomad_go_bg.shape[0]}')
print(f'Shape before filtering from duplicates: {filtered_data.shape[0]}')

Number of duplicates: 184
Shape before filtering from duplicates: 57874
Shape before filtering from duplicates: 57783


Check result

In [73]:
set(annot['gene_name']) == set(filtered_data['gene_name'])

True

Fun Fact: in human genome annotation some genes have same start/end/gene name, but different `gene_type`

In [72]:
duplicates_mask = annot.duplicated(subset=['chr', 'start', 'end', 'gene_name'], keep=False)
annot[duplicates_mask]

,chr,start,end,strand,gene_name,gene_type
27050,chr19,36208920,36229779,+,KMT2B,processed_transcript
27051,chr19,36208920,36229779,+,KMT2B,protein_coding


Merge with GTEx data

In [92]:
path_to_gtex_v8 = os.path.join(current_dir, '..', 'data', 'gtex_median_gene_expression_by_tissue', 'gtex_counts_v8.tsv')
gtex = pd.read_csv(path_to_gtex_v8, sep='\t')

In [ ]:
all_data = pd.merge(filtered_data, gtex, left_on='gene_name', right_on='Description', how='left')
all_data.rename(columns={'Name': 'ENSG_from_GTEx', 'interactors': 'interactors_bg', 
                  'n_interactions': 'n_interactions'}, inplace=True)

In [107]:
all_data.columns

Index(['chr', 'start', 'end', 'gene_name', 'gene_type', 'gene_len',
       'gene_len_500', 'RS_value', 'RS_500_value', 'phast_value',
       'phast_500_value', 'phylo_value', 'phylo_500_value', 'iHS_GBR_max_500',
       'expected_iHS', 'iHS_snps_per_gene', 'pval_iHS_max', 'pval_iHS_max_fdr',
       'DRC150', 'SDS', 'DRC150_signif_snp_count', 'gene', 'chromosome',
       'oe_lof_upper', 'pLI', 'mis_z', 'GO_BP_Count', 'GO_MF_Count',
       'interactors_bg', 'n_interactions', 'ENSG_from_GTEx', 'Description',
       'median_expr_all_tissues', 'max_expr_all_tissues',
       'num_tissues_at_least_5_TPM'],
      dtype='object')

In [108]:
#save all data
path_to_all_data = (os.path.join(current_dir, '..', 'data', 'merged_data',
                                           'css_project_metrics.tsv'))

all_data[['chr', 'start', 'end', 'gene_name', 'gene_type', 'gene_len',
       'gene_len_500', 'RS_value', 'RS_500_value', 'phast_value',
       'phast_500_value', 'phylo_value', 'phylo_500_value', 'iHS_GBR_max_500',
       'expected_iHS', 'iHS_snps_per_gene', 'pval_iHS_max', 'pval_iHS_max_fdr',
       'DRC150', 'SDS', 'DRC150_signif_snp_count', 'oe_lof_upper', 'pLI', 'mis_z', 
       'GO_BP_Count', 'GO_MF_Count', 'interactors_bg', 'n_interactions', 
       'ENSG_from_GTEx', 'median_expr_all_tissues', 'max_expr_all_tissues',
       'num_tissues_at_least_5_TPM']].to_csv(path_to_all_data, sep='\t', header=True, index=False)

# Filter SNPs for DRC150

Obtained data merged with df

In [100]:
drc150 = pd.read_csv(os.path.join(current_dir, '..', 'data', 'drc150_positive_selection', 'DRC_150.180813.bed'),
                     sep='\t', names=['chr', 'start', 'end', 'drc150', 'pval'])
drc150['chr'] = drc150['chr'].apply(lambda x: f'chr{x}')

In [106]:
#fdr correction 
drc150['pval_fdr'] = stats.false_discovery_control(ps=drc150['pval'], method='bh')

#filtering
drc150_filtered = drc150.query('pval_fdr < 0.05')

In [ ]:
drc150_filtered[['chr', 'start', 'end', 'pval_fdr']].to_csv(os.path.join(current_dir, '..', 'data', 'drc150_positive_selection', 'drc150_signif_snps.bed'), sep='\t', index=False)

After count significiant SNPs per gene with bedtools

# Count protein-protein interactions on BioGRID data

File Version: 4.4.245

[Link to download](https://downloads.thebiogrid.org/BioGRID/Release-Archive/BIOGRID-4.4.245/)

Info about data and columns:

Stark C, Breitkreutz BJ, Reguly T, Boucher L, Breitkreutz A, Tyers M. BioGRID: A General Repository for Interaction Datasets.
Nucleic Acids Res. Jan1;34:D535-9.

A.) INTERACTOR_A			Unique ID for Interacting Partner A

B.) INTERACTOR_B			Unique ID for Interacting Partner B

C.) OFFICIAL_SYMBOL_FOR_A

D.) OFFICIAL_SYMBOL_FOR_B

E.) ALIASES_FOR_A			List of common names for geneA, separated by '|'

F.) ALIASES_FOR_B			List of common names for geneB, separated by '|'

G.) EXPERIMENTAL_SYSTEM			System in which the interaction was shown

H.) SOURCE				Author/s of the interaction

I.) PUBMED_ID				PubMed_ID of the paper, separated by ';'

J.) ORGANISM_A_ID			NCBI ID of Gene A Organism

K.) ORGANISM_B_ID			NCBI ID of Gene B Organism

Code from the [pleiotropy research project](https://github.com/Ibre-research/pleiotropy_analysis/blob/main/preliminary_analysis.ipynb) was used as a code example

In [5]:
#due to the large size of the data files for this code is not in the repository
path_to_biogrid = os.path.abspath(os.path.join(current_dir, '..', 'data', 'biogrid',
                                            'biogrid_organism_Hs_4.4.245.txt'))

bg = pd.read_csv(path_to_biogrid, sep='\t', low_memory=False)

In [10]:
#due to the large size of the data files for this code is not in the repository
#filter data for human (Taxonomy ID 9606)
bg_filtered = bg[(bg['ORGANISM_A_ID'] == 9606) & (bg['ORGANISM_B_ID'] == 9606)]

interactions = defaultdict(lambda: {
    'gene_name': None,
    'interactors': set(),
    'n_interactions': 0
})

for _, row in bg_filtered.iterrows():
    gene_a = row['OFFICIAL_SYMBOL_A']
    gene_b = row['OFFICIAL_SYMBOL_B']
    
    interactions[gene_a]['gene_name'] = gene_a 
    interactions[gene_a]['interactors'].add(gene_b)
    interactions[gene_a]['n_interactions'] += 1
    
    interactions[gene_b]['gene_name'] = gene_b  
    interactions[gene_b]['interactors'].add(gene_a)
    interactions[gene_b]['n_interactions'] += 1

interactions = pd.DataFrame.from_dict(interactions, orient='index')
interactions = interactions.reset_index(drop=True)
interactions

,gene_name,interactors,n_interactions
0,MAP2K4,"{ZDHHC17, MAP3K2, MAP2K4, SPAG9, GEMIN5, CNDP2...",126
1,FLNC,"{MAP2K4, IARS, WDR77, PIAS1, MARS, FASN, MECP2...",271
2,MYPN,"{KIF14, NEBL, BAG6, NEB, ACTN3, ANKRD1, PALLD,...",37
3,ACTN2,"{LDB3, ACTN3, ODF3B, TPM2, ZC2HC1C, GRIN1, PPP...",192
4,ACVR1,"{FKBP1A, OSTF1, PSMD14, ERBB2, INHBA, ACVR1B, ...",99
...,...,...,...
20220,IGKV1-9,{SHANK3},1
20221,TRBV10-2,{SHANK3},1
20222,IGKV3D-15,{SHANK3},1
20223,NPY6R,{CHD8},1


In [16]:
path_to_bg_inter = os.path.abspath(os.path.join(current_dir, '..', 'data', 'biogrid',
                                            'biogrid_n_interactions_per_gene.tsv'))
interactions.to_csv(path_to_bg_inter, index=False, sep='\t')

# Preparing of GTEx data

Data Version: 8

In [80]:
path_to_gtex8 = os.path.abspath(os.path.join(current_dir, '..', 'data', 'gtex_median_gene_expression_by_tissue',
                                            'gtex_v8_gene_median_tpm.gct'))

#ENSG for chrM genes
path_to_ensg_chrM = os.path.abspath(os.path.join(current_dir, '..', 'data', 'annotation',
                                            'ensg_chrM_hg19.tsv'))

gtex8 = pd.read_csv(path_to_gtex8, sep='\t')
ensg_chrM = pd.read_csv(path_to_ensg_chrM, sep='\t', names=['chr', 'start', 'end', 'gene_name', 'ENSG', 'gene_type'])

In [81]:
#median
gtex8['median_expr_all_tissues'] = gtex8.iloc[:, 2:70].median(axis=1)

#max
gtex8['max_expr_all_tissues'] = gtex8.iloc[:, 2:70].max(axis=1)

#number of tissues in which a given gene is expressed at the level of at least 5 TPM
gtex8['num_tissues_at_least_5_TPM'] = gtex8.iloc[:, 2:70].apply(lambda x: (x >= 5).sum(), axis=1)

In [86]:
#code from pleiotropy project 
#remove duplicates
duplicates = gtex8[gtex8.duplicated(subset=['Description'], keep=False)]

# remove duplicates with 0 expression related to _PAR_Y
for index, row in duplicates.iterrows():
    if (row['median_expr_all_tissues'] == 0) and (row['max_expr_all_tissues'] == 0) and (row['num_tissues_at_least_5_TPM'] == 0):
        gtex8 = gtex8.drop(index)

# delete all duplicates with double ENSG
gtex8.drop_duplicates(subset=['Description'], keep=False, inplace=True)

In [88]:
#delete chrM genes
gtex8_filtered = gtex8[~gtex8['Description'].str.startswith('MT-')]
print(f'Shape of gtex8_filtered from chrM genes {gtex8_filtered.shape}')

Shape of gtex8_filtered from chrM genes (54428, 59)


In [89]:
print(f'Shape of gtex8 after all filtering {gtex8_filtered.shape}')

Shape of gtex8 after all filtering (54428, 59)


In [90]:
#get list of names from column Description
gtex_genes = gtex8['Description'].to_list()
ensg_in_descr_column = [name for name in gtex_genes if name.startswith('ENSG')]
print(f'Number ENSG ID in gtex_filtered df: {len(ensg_in_descr_column)}')

Number ENSG ID in gtex_filtered df: 0


In [91]:
#save data
path_to_gtex_v8 = os.path.join(current_dir, '..', 'data', 'gtex_median_gene_expression_by_tissue', 'gtex_counts_v8.tsv')
gtex8[['Name', 'Description', 'median_expr_all_tissues', 'max_expr_all_tissues', 
       'num_tissues_at_least_5_TPM']].to_csv(path_to_gtex_v8, sep='\t', index=False)

##### Thinking about column `Description`

In this column there are ENSG ID and gene names in GTEx v.10

Search what is with ENSG ID

Use `mygene` package to get info about ENSG ID from column `Description`

In [115]:
#get list of names from column Description
path_to_gtex10 = os.path.join(current_dir, '..', 'data', 'gtex_median_gene_expression_by_tissue',
                                            'gtex_v10_gene_median_tpm.gct')

gtex_raw = pd.read_csv(path_to_gtex10, sep='\t')
gtex_genes = gtex_raw['Description'].to_list()
ensg_in_descr_column = [name for name in gtex_genes if name.startswith('ENSG')]
print(f'Number ENSG ID in gtex_filtered df: {len(ensg_in_descr_column)}')

Number ENSG ID in gtex_filtered df: 19118


In [116]:
#get data about ENSG from column Description
import mygene
mg = mygene.MyGeneInfo()
result = mg.querymany(ensg_in_descr_column, scopes="ensembl.gene", fields="symbol,name", species="human")

result_dict = {}
for gene in result:
    result_dict[f'{gene['query']}'] = (f'{gene.get('symbol', 'N/A')}', f'{gene.get('name', 'N/A')}')

Input sequence provided is already in string format. No operation performed
Input sequence provided is already in string format. No operation performed
29 input query terms found dup hits:	[('ENSG00000228044', 2), ('ENSG00000226506', 2), ('ENSG00000261600', 2), ('ENSG00000234162', 2), ('E
988 input query terms found no hit:	['ENSG00000238009', 'ENSG00000288531', 'ENSG00000230699', 'ENSG00000241180', 'ENSG00000236948', 'ENS


In [117]:
#make df for obtained from mygene data
ensg_list = list()
gene_names = list()
description = list()

for ensg, gene in result_dict.items():
    ensg_list.append(ensg)
    gene_names.append(gene[0])
    description.append(gene[1])

df_ensg_in_descr_col = pd.DataFrame({'ENSG': ensg_list,
                                     'gene_name': gene_names,
                                     'description': description})

#df_ensg_in_descr_col_no_na = df_ensg_in_descr_col.query('gene_name != "N/A" and description != "N/A"')
#df_ensg_in_descr_col_no_na.to_csv('gtex_ensg_in_descr_col.tsv', sep='\t', index=False)